<a href="https://colab.research.google.com/github/zqian/colab_scripts/blob/main/canvas_course_assignment_submissions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install canvas api
!pip install canvasapi

In [ ]:
import pandas as pd
# Import the Canvas class
from canvasapi import Canvas
from canvasapi.tab import Tab
from getpass import getpass

In [ ]:
# Canvas API URL
API_URL = "canvas_url"
# Canvas API key (generated from settings)
API_KEY = getpass('Enter your API KEY:')

# Initialize a new Canvas object
canvas_api = Canvas(API_URL, API_KEY)


In [ ]:
account_id = ACCOUNT_ID
course_id = COURSE_ID

In [ ]:
account = canvas_api.get_account(account_id)
course = canvas_api.get_course(course_id)
assignments = course.get_assignments()

In [ ]:
column_header_list = ['Student', 'ID', 'SIS User ID', 'SIS Login ID', 'Section']
for assignment in assignments:
  assignment_entry = assignment.name + "(" + str(assignment.id) + ")"
  column_header_list.append(assignment_entry)

print(column_header_list)
df = pd.DataFrame(columns=column_header_list)


In [ ]:
# get all class enrollment
for enrollment in course.get_enrollments():
    user_id = enrollment.user_id
    user_sis_id = enrollment.sis_user_id
    course_section_id = enrollment.course_section_id
    section = canvas_api.get_section(course_section_id)

    # only get active enrollment
    if enrollment.enrollment_state == 'active':
      user = canvas_api.get_user(int(user_id))
      user_dict = {
          'Student': user.sortable_name,
          'ID': user_id, 
          'SIS User ID': user_sis_id,
          'SIS Login ID': user.login_id, 
          'Section':section.name
      }
      for assignment in assignments:
        # initialize status to empty string
        assignment_entry = assignment.name + "(" + str(assignment.id) + ")"
        user_dict[assignment_entry] = ""

      # append to the end of dataframe
      df_user = pd.DataFrame([user_dict])
      df = pd.concat([df, df_user], axis=0)

In [ ]:
# now update the submission grade/status
for assignment in assignments:

  assignment_entry = assignment.name + "(" + str(assignment.id) + ")"
  print(assignment_entry)
  submissions = assignment.get_submissions()
  for submission in submissions:
    # show submission grade if available, otherwise show submission workflow status
    status = submission.grade if (submission.grade is not None and len(submission.grade)>0) else submission.workflow_state
    user_dict[assignment.name] = status

    user_id = submission.user_id

    # find the user row
    df.loc[df['ID'] == user_id, assignment_entry] = status

      

In [ ]:
# output csv file
df.to_csv('grades_output_new.csv') 
files.download('grades_output_new.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>